In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import subprocess
import time
from tqdm.notebook import tqdm, trange


# Overview
This notebook is used to generate some sweeps of `wcalc` results compared to electrostatic based results obtained via using `FreeFem++` to solve the 2-D Laplace's equation subject to the boundary conditions imposed by the structure cross section and an applied test voltage.

*NOTE* This notebook is still very much a WIP and it may be the case that the code is converted to a standalone wrapper for generating lots of data with a different program for plotting the data.

In [ ]:
# Until wcalc learns how to more directly interface with python, use
# the command line interface
stdio_wcalc = Path("../../stdio-wcalc/stdio-wcalc")

def coplanar_calc_wcalc(W, S, H, Tmet, er, gnd_flag):
    gnd = 1 if gnd_flag else 0
    # [z0, keff, len loss, L, R, C, G, lc, ld, deltal, skindepth] =
    #   coplanar_calc(w,s,h,l,tmet,rho,rough,er,tand,with_ground,f);
    tmps = f"coplanar_calc {W} {S} {H} 1.0 {Tmet} 1e-8 0 {er} 0 {gnd} 0"
    rslt = subprocess.run([stdio_wcalc], stdout=subprocess.PIPE, input=tmps.encode("utf-8"))
    tmps = rslt.stdout.decode("utf-8")
    tmpa = tmps.split()
    z0 = tmpa[0]
    k = tmpa[1]

    return k, z0

def microstrip_calc_wcalc(W, H, Tmet, er):
    # [z0, keff, elen, loss, L, R, C, G, lc, ld, deltal, depth] =
    #   microstrip_calc(w,h,l,tmet,rho,rough,er,tand,f);
    tmps = f"microstrip_calc {W} {H} 1.0 {Tmet} 1e-8 0 {er} 0 0"
    rslt = subprocess.run([stdio_wcalc], stdout=subprocess.PIPE, input=tmps.encode("utf-8"))
    tmps = rslt.stdout.decode("utf-8")
    tmpa = tmps.split()
    z0 = tmpa[0]
    k = tmpa[1]

    return k, z0

def stripline_calc_wcalc(W, H, Tmet, er):
    # [z0, elen, loss, L, R, C, G, lc, ld, deltal, depth] =
    #   stripline_calc(w,h,l,tmet,rho,rough,er,tand,f);
    tmps = f"stripline_calc {W} {H} 1.0 {Tmet} 1e-8 0 {er} 0 0"
    rslt = subprocess.run([stdio_wcalc], stdout=subprocess.PIPE, input=tmps.encode("utf-8"))
    tmps = rslt.stdout.decode("utf-8")
    tmpa = tmps.split()
    z0 = tmpa[0]

    return z0


free_fem = "FreeFem++"
def run_fem(data, name):
    fem_tmp = Path("./temp.dat")
    with open(fem_tmp, "w") as fp:
        fp.write(data)
        fp.write("\n")
    
    args = [free_fem]
    
    # don't wait on graphics at end
    args.append("-nowait")
    
    # in fact, don't even display graphics
    args.append("-nw")
    
    # don't write to stdout
    #args.append("-ne")

    args.append("-f")
    args.append(f"{name}.edp")
    
    args.append("-df")
    args.append(fem_tmp)
    
    rslt = subprocess.run(args, stdout=subprocess.PIPE)
    if rslt.returncode != 0:
        print("FEM failed:")
        print(args)
        print(rslt.stdout.decode("utf-8"))
        return None
    else:
        outf = Path(f"./{name}.txt")
        with open(outf) as fp:
            lines = fp.readlines()
        return lines[1]
    
def coplanar_calc_fem(W, S, H, Tmet, er, gnd_flag):
    gnd = 1 if gnd_flag else 0
    tmps = f"{W} {S} {H} {Tmet} {er} {gnd}"
    rslt = run_fem(tmps, "coplanar")
    if rslt is not None:
        tmp = rslt.split()
        z0 = tmp[-1]
        k = tmp[-2]
    else:
        # hate using a magic value instead of None
        # but it makes it clear in plots when something is wrong
        z0 = k = 0
        
    return k, z0

def microstrip_calc_fem(W, H, Tmet, er):
    tmps = f"{W} {H} {Tmet} {er}"
    rslt = run_fem(tmps, "microstrip")
    if rslt is not None:
        tmp = rslt.split()
        z0 = tmp[-1]
        k = tmp[-2]
    else:
        # hate using a magic value instead of None
        # but it makes it clear in plots when something is wrong
        z0 = k = 0
        
    return k, z0

def stripline_calc_fem(W, H, Tmet, er):
    tmps = f"{W} {H} {Tmet} {er}"
    rslt = run_fem(tmps, "stripline")
    if rslt is not None:
        tmp = rslt.split()
        z0 = tmp[-1]
    else:
        # hate using a magic value instead of None
        # but it makes it clear in plots when something is wrong
        z0 = 0
        
    return z0

def fmtnum(v, dec=1):
    """Convert a decimal number to a file name friendly string
    Args:
    v - number
    dec - how many decimal places to preserve.
    
    fmtnum(3.123, 2) will give "3p12"
    """
    
    # first round to the number of desired places
    sf = 10**dec
    vs = round(v*sf)

    # find the integer part
    i = vs // sf
    
    # find the fractional part
    f = vs - i*sf
    
    # and now the string
    r = f"{i}p{f:=0{dec}d}"

    return r


er = 4.8
Tmet = 1
W = 2.5
S = 50
H = 10
useGnd = True

if not False:
    print(f"width={W}, space={S}, height={H}, thickness={Tmet}, er={er}")
    (k,z) = coplanar_calc_fem(W, S, H, Tmet, er, useGnd)
    print(f"FEM gnd={useGnd} coplanar z0 = {z}, keff = {k}")

    (k,z) = coplanar_calc_wcalc(W, S, H, Tmet, er, useGnd)
    print(f"wcalc gnd={useGnd} coplanar z0 = {z}, keff = {k}")

if not False:
    (k,z) = microstrip_calc_fem(W, H, Tmet, er)
    print(f"FEM Extracted microstrip z0 = {z}, keff = {k}")

    (k,z) = microstrip_calc_wcalc(W, H, Tmet, er)
    print(f"wcalc Extracted microstrip z0 = {z}, keff = {k}")

if False:
    z = stripline_calc_fem(10, 10, 1, 4.8)
    print(f"FEM Extracted stripline z0 = {z}")

    z = stripline_calc_wcalc(10, 10, 1, 4.8)
    print(f"wcalc Extracted stripline z0 = {z}")



In [ ]:
H = 10.0
W = 5.0
S = 5.0
Tmet = 1.0
gndFlag = False

short = False
update_calc = not False

all_er = np.array([1.0, 2.55, 4.8, 9.6])
all_tmet = np.array([0, 1.0, 2.0, 5.0])
num_w = 25

if short:
    all_er = np.array([1.0, 4.8])
    all_tmet = np.array([0, 2.5, 5.0])
    num_w = 5
    
all_w = np.logspace(np.log10(0.1*H), np.log10(10.0*H), num_w)
all_s = np.logspace(np.log10(0.1*H), np.log10(10.0*H), num_w)

sty_wc = {
    #"color" : "b",
    "linestyle" : "-"
}
sty_fem = {
    #"color" : "r",
    "linestyle" : "-."
}
colors = "rbky"

num_er = all_er.shape[0]
num_tmet = all_tmet.shape[0]
num_w = all_w.shape[0]
shp = [num_er, num_tmet, num_w]

save_dir = Path("./results")
save_dir.mkdir(mode=0o755, parents=True, exist_ok=True)
dpi = 150


In [ ]:
update_calc = False
gndFlag = False
gndStr2 = "gnd" if gndFlag else "nognd"
npz = save_dir / f"coplanar_{gndStr2}.npz"

if update_calc:
    keff_coplanar_wc = np.zeros(shp)
    z0_coplanar_wc = np.zeros(shp)
    keff_coplanar_fem = np.zeros(shp)
    z0_coplanar_fem = np.zeros(shp)
else:
    data = np.load(npz)
    keff_coplanar_wc = data["keff_coplanar_wc"]
    keff_coplanar_fem = data["keff_coplanar_fem"]
    z0_coplanar_wc = data["z0_coplanar_wc"]
    z0_coplanar_fem = data["z0_coplanar_fem"]

if update_calc:
    for i_er in trange(num_er, desc="Er"):
        er = all_er[i_er]    
        for i_tmet in trange(num_tmet, desc="Tmet", leave=None):
            Tmet = all_tmet[i_tmet]
            for i_w in trange(num_w, desc="W", leave=None):
                Wx = all_w[i_w]

                (keff , z0) = coplanar_calc_wcalc(Wx, S, H, Tmet, er, gndFlag)
                keff_coplanar_wc[i_er, i_tmet, i_w] = keff
                z0_coplanar_wc[i_er, i_tmet, i_w] = z0

                (keff , z0) = coplanar_calc_fem(Wx, S, H, Tmet, er, gndFlag)
                keff_coplanar_fem[i_er, i_tmet, i_w] = keff
                z0_coplanar_fem[i_er, i_tmet, i_w] = z0
    data = {
        "keff_coplanar_wc" : keff_coplanar_wc,
        "keff_coplanar_fem" : keff_coplanar_fem,
        "z0_coplanar_wc" : z0_coplanar_wc,
        "z0_coplanar_fem" : z0_coplanar_fem,
    }
    np.savez_compressed(npz, **data)
    
gndStr = "With" if gndFlag else "Without"
gndStr += " bottom GND"
for i_er in range(num_er):
    er = all_er[i_er]

    figk = plt.figure()
    ax_k = plt.subplot(111)
    ax_k.set_xscale("log")
    ax_k.grid()
    ax_k.set_title(f"Coplanar Effective Dielectric Constant\ner = {er}, S/H={S/H}, {gndStr}")
    ax_k.set_xlabel("Width/Height")
    ax_k.set_ylabel("keff")

    figz = plt.figure()
    ax_z = plt.subplot(111)
    ax_z.set_xscale("log")
    ax_z.grid()
    ax_z.set_title(f"Coplanar Characteristic Impedance\ner = {er}, S/H={S/H} {gndStr}")
    ax_z.set_xlabel("Width/Height")
    ax_z.set_ylabel("Z0 [Ohms]")
    
    for i_tmet in range(num_tmet):
        Tmet = all_tmet[i_tmet]
        sty_wc["color"] = sty_fem["color"] = colors[i_tmet % len(colors)]

        x = all_w / H
        ax_k.plot(x, keff_coplanar_wc[i_er, i_tmet, :], label=f"WC: er={er}, Tmet={Tmet}", **sty_wc)
        ax_z.plot(x, z0_coplanar_wc[i_er, i_tmet, :], label=f"WC: er={er}, Tmet={Tmet}", **sty_wc)
        ax_k.plot(x, keff_coplanar_fem[i_er, i_tmet, :], label=f"FEM: er={er}, Tmet={Tmet}", **sty_fem)
        ax_z.plot(x, z0_coplanar_fem[i_er, i_tmet, :], label=f"FEM: er={er}, Tmet={Tmet}", **sty_fem)
    ax_k.legend()
    ax_z.legend()
    pout = save_dir / f"coplanar_{gndStr2}_keff.png"
    figk.savefig(str(pout), dpi=dpi, bbox_inches="tight")
    pout = save_dir / f"coplanar_{gndStr2}_z0.png"
    figz.savefig(str(pout), dpi=dpi, bbox_inches="tight")



In [ ]:
update_calc = False
gndFlag = True
gndStr2 = "gnd" if gndFlag else "nognd"
npz = save_dir / f"coplanar_{gndStr2}.npz"

if update_calc:
    keff_coplanar_gnd_wc = np.zeros(shp)
    z0_coplanar_gnd_wc = np.zeros(shp)
    keff_coplanar_gnd_fem = np.zeros(shp)
    z0_coplanar_gnd_fem = np.zeros(shp)
else:
    data = np.load(npz)
    keff_coplanar_gnd_wc = data["keff_coplanar_gnd_wc"]
    keff_coplanar_gnd_fem = data["keff_coplanar_gnd_fem"]
    z0_coplanar_gnd_wc = data["z0_coplanar_gnd_wc"]
    z0_coplanar_gnd_fem = data["z0_coplanar_gnd_fem"]

if update_calc:
    for i_er in trange(num_er, desc="Er"):
        er = all_er[i_er]    
        for i_tmet in trange(num_tmet, desc="Tmet", leave=None):
            Tmet = all_tmet[i_tmet]
            for i_w in trange(num_w, desc="W", leave=None):
                Wx = all_w[i_w]

                (keff , z0) = coplanar_calc_wcalc(Wx, S, H, Tmet, er, gndFlag)
                keff_coplanar_gnd_wc[i_er, i_tmet, i_w] = keff
                z0_coplanar_gnd_wc[i_er, i_tmet, i_w] = z0

                (keff , z0) = coplanar_calc_fem(Wx, S, H, Tmet, er, gndFlag)
                keff_coplanar_gnd_fem[i_er, i_tmet, i_w] = keff
                z0_coplanar_gnd_fem[i_er, i_tmet, i_w] = z0
    data = {
        "keff_coplanar_gnd_wc" : keff_coplanar_gnd_wc,
        "keff_coplanar_gnd_fem" : keff_coplanar_gnd_fem,
        "z0_coplanar_gnd_wc" : z0_coplanar_gnd_wc,
        "z0_coplanar_gnd_fem" : z0_coplanar_gnd_fem,
    }
    np.savez_compressed(npz, **data)
        
gndStr = "With" if gndFlag else "Without"
gndStr += " bottom GND"
for i_er in range(num_er):
    er = all_er[i_er]

    figk = plt.figure()
    ax_k = plt.subplot(111)
    ax_k.set_xscale("log")
    ax_k.grid()
    ax_k.set_title(f"Coplanar Effective Dielectric Constant\ner = {er}, S/H={S/H}, {gndStr}")
    ax_k.set_xlabel("Width/Height")
    ax_k.set_ylabel("keff")

    figz = plt.figure()
    ax_z = plt.subplot(111)
    ax_z.set_xscale("log")
    ax_z.grid()
    ax_z.set_title(f"Coplanar Characteristic Impedance\ner = {er}, S/H={S/H} {gndStr}")
    ax_z.set_xlabel("Width/Height")
    ax_z.set_ylabel("Z0 [Ohms]")
    
    for i_tmet in range(num_tmet):
        Tmet = all_tmet[i_tmet]
        sty_wc["color"] = sty_fem["color"] = colors[i_tmet % len(colors)]

        x = all_w / H
        ax_k.plot(x, keff_coplanar_gnd_wc[i_er, i_tmet, :], label=f"WC: er={er}, Tmet={Tmet}", **sty_wc)
        ax_z.plot(x, z0_coplanar_gnd_wc[i_er, i_tmet, :], label=f"WC: er={er}, Tmet={Tmet}", **sty_wc)
        ax_k.plot(x, keff_coplanar_gnd_fem[i_er, i_tmet, :], label=f"FEM: er={er}, Tmet={Tmet}", **sty_fem)
        ax_z.plot(x, z0_coplanar_gnd_fem[i_er, i_tmet, :], label=f"FEM: er={er}, Tmet={Tmet}", **sty_fem)
    ax_k.legend()
    ax_z.legend()
    pout = save_dir / f"coplanar_{gndStr2}_keff.png"
    figk.savefig(str(pout), dpi=dpi, bbox_inches="tight")
    pout = save_dir / f"coplanar_{gndStr2}_z0.png"
    figz.savefig(str(pout), dpi=dpi, bbox_inches="tight")


In [ ]:
update_calc = False
npz = save_dir / f"microstrip.npz"

if update_calc:
    keff_microstrip_wc = np.zeros(shp)
    z0_microstrip_wc = np.zeros(shp)
    keff_microstrip_fem = np.zeros(shp)
    z0_microstrip_fem = np.zeros(shp)
else:
    data = np.load(npz)
    keff_microstrip_wc = data["keff_microstrip_wc"]
    keff_microstrip_fem = data["keff_microstrip_fem"]
    z0_microstrip_wc = data["z0_microstrip_wc"]
    z0_microstrip_fem = data["z0_microstrip_fem"]

if update_calc:
    for i_er in trange(num_er, desc="Er"):
        er = all_er[i_er]    
        for i_tmet in trange(num_tmet, desc="Tmet", leave=None):
            Tmet = all_tmet[i_tmet]
            for i_w in trange(num_w, desc="W", leave=None):
                Wx = all_w[i_w]

                (keff , z0) = microstrip_calc_wcalc(Wx, H, Tmet, er)
                keff_microstrip_wc[i_er, i_tmet, i_w] = keff
                z0_microstrip_wc[i_er, i_tmet, i_w] = z0

                (keff , z0) = microstrip_calc_fem(Wx, H, Tmet, er)
                keff_microstrip_fem[i_er, i_tmet, i_w] = keff
                z0_microstrip_fem[i_er, i_tmet, i_w] = z0
    data = {
        "keff_microstrip_wc" : keff_microstrip_wc,
        "keff_microstrip_fem" : keff_microstrip_fem,
        "z0_microstrip_wc" : z0_microstrip_wc,
        "z0_microstrip_fem" : z0_microstrip_fem,
    }
    np.savez_compressed(npz, **data)        

for i_er in range(num_er):
    er = all_er[i_er]

    figk = plt.figure()
    ax_k = plt.subplot(111)
    ax_k.set_xscale("log")
    ax_k.grid()
    ax_k.set_title(f"Microstrip Effective Dielectric Constant\ner = {er}")
    ax_k.set_xlabel("Width/Height")
    ax_k.set_ylabel("keff")

    figz = plt.figure()
    ax_z = plt.subplot(111)
    ax_z.set_xscale("log")
    ax_z.grid()
    ax_z.set_title(f"Microstrip Characteristic Impedance\ner = {er}")
    ax_z.set_xlabel("Width/Height")
    ax_z.set_ylabel("Z0 [Ohms]")
    
    for i_tmet in range(num_tmet):
        Tmet = all_tmet[i_tmet]
        sty_wc["color"] = sty_fem["color"] = colors[i_tmet % len(colors)]

        x = all_w / H
        ax_k.plot(x, keff_microstrip_wc[i_er, i_tmet, :], label=f"WC: er={er}, Tmet={Tmet}", **sty_wc)
        ax_z.plot(x, z0_microstrip_wc[i_er, i_tmet, :], label=f"WC: er={er}, Tmet={Tmet}", **sty_wc)
        ax_k.plot(x, keff_microstrip_fem[i_er, i_tmet, :], label=f"FEM: er={er}, Tmet={Tmet}", **sty_fem)
        ax_z.plot(x, z0_microstrip_fem[i_er, i_tmet, :], label=f"FEM: er={er}, Tmet={Tmet}", **sty_fem)
    ax_k.legend()
    ax_z.legend()
    pout = save_dir / "microstrip_keff.png"
    figk.savefig(str(pout), dpi=dpi, bbox_inches="tight")
    pout = save_dir / "microstrip_z0.png"
    figz.savefig(str(pout), dpi=dpi, bbox_inches="tight")


# Stripline

We expect very good agreement in the stripline case.  There are closed form solutions available and ignoring the finite conductance, true TEM propagation is supported.

In [ ]:
update_calc = False
npz = save_dir / f"stripline.npz"

if update_calc:
    z0_stripline_wc = np.zeros(shp)
    z0_stripline_fem = np.zeros(shp)
else:
    data = np.load(npz)
    z0_stripline_wc = data["z0_stripline_wc"]
    z0_stripline_fem = data["z0_stripline_fem"]

if update_calc:
    for i_er in trange(num_er, desc="Er"):
        er = all_er[i_er]    
        for i_tmet in trange(num_tmet, desc="Tmet", leave=None):
            Tmet = all_tmet[i_tmet]
            for i_w in trange(num_w, desc="W", leave=None):
                Wx = all_w[i_w]

                z0 = stripline_calc_wcalc(Wx, H, Tmet, er)
                z0_stripline_wc[i_er, i_tmet, i_w] = z0

                z0 = stripline_calc_fem(Wx, H, Tmet, er)
                z0_stripline_fem[i_er, i_tmet, i_w] = z0
    data = {
        "z0_stripline_wc" : z0_stripline_wc,
        "z0_stripline_fem" : z0_stripline_fem,
    }
    np.savez_compressed(npz, **data)        
        

for i_er in range(num_er):
    er = all_er[i_er]

    figz = plt.figure()
    ax_z = plt.subplot(111)
    ax_z.set_xscale("log")
    ax_z.grid()
    ax_z.set_title(f"Stripline Characteristic Impedance\ner = {er}")
    ax_z.set_xlabel("Width/Height")
    ax_z.set_ylabel("Z0 [Ohms]")
    
    for i_tmet in range(num_tmet):
        Tmet = all_tmet[i_tmet]
        sty_wc["color"] = sty_fem["color"] = colors[i_tmet % len(colors)]

        x = all_w / H
        ax_z.plot(x, z0_stripline_wc[i_er, i_tmet, :], label=f"WC: er={er}, Tmet={Tmet}", **sty_wc)
        ax_z.plot(x, z0_stripline_fem[i_er, i_tmet, :], label=f"FEM: er={er}, Tmet={Tmet}", **sty_fem)
    ax_z.legend()
    er_str = fmtnum(er, 1)
    pout = save_dir / "stripline_z0_er{er_str}.png"
    figz.savefig(str(pout), dpi=dpi, bbox_inches="tight")
        

In [ ]:
update_calc = False
update_fem = False
gndFlag = True
gndStr2 = "gnd" if gndFlag else "nognd"
npz = save_dir / f"coplanar_{gndStr2}2.npz"

if update_calc:
    keff_coplanar_gnd2_wc = np.zeros(shp)
    z0_coplanar_gnd2_wc = np.zeros(shp)
    if update_fem:
        keff_coplanar_gnd2_fem = np.zeros(shp)
        z0_coplanar_gnd2_fem = np.zeros(shp)

    tmp = [num_er, num_tmet]
    keff_coplanar_gnd2_mswc = np.zeros(tmp)
    z0_coplanar_gnd2_mswc = np.zeros(tmp)

    if update_fem:
        keff_coplanar_gnd2_msfem = np.zeros(tmp)
        z0_coplanar_gnd2_msfem = np.zeros(tmp)

else:
    data = np.load(npz)
    keff_coplanar_gnd2_wc = data["keff_coplanar_gnd2_wc"]
    keff_coplanar_gnd2_fem = data["keff_coplanar_gnd2_fem"]
    z0_coplanar_gnd2_wc = data["z0_coplanar_gnd2_wc"]
    z0_coplanar_gnd2_fem = data["z0_coplanar_gnd2_fem"]

    keff_coplanar_gnd2_mswc = data["keff_coplanar_gnd2_mswc"]
    keff_coplanar_gnd2_msfem = data["keff_coplanar_gnd2_msfem"]
    z0_coplanar_gnd2_mswc = data["z0_coplanar_gnd2_mswc"]
    z0_coplanar_gnd2_msfem = data["z0_coplanar_gnd2_msfem"]

W = 0.5*H
if update_calc:
    for i_er in trange(num_er, desc="Er"):
        er = all_er[i_er]    
        for i_tmet in trange(num_tmet, desc="Tmet", leave=None):
            Tmet = all_tmet[i_tmet]
            
            # microstrip limit of very large S
            
            (keff , z0) = microstrip_calc_wcalc(W, H, Tmet, er)
            keff_coplanar_gnd2_mswc[i_er, i_tmet] = keff
            z0_coplanar_gnd2_mswc[i_er, i_tmet] = z0

            if update_fem:
                (keff , z0) = microstrip_calc_fem(W, H, Tmet, er)
                keff_coplanar_gnd2_msfem[i_er, i_tmet] = keff
                z0_coplanar_gnd2_msfem[i_er, i_tmet] = z0
            
            for i_s in trange(num_w, desc="S", leave=None):
                Sx = all_s[i_s]

                (keff , z0) = coplanar_calc_wcalc(W, Sx, H, Tmet, er, gndFlag)
                keff_coplanar_gnd2_wc[i_er, i_tmet, i_s] = keff
                z0_coplanar_gnd2_wc[i_er, i_tmet, i_s] = z0
                
                if update_fem:
                    (keff , z0) = coplanar_calc_fem(W, Sx, H, Tmet, er, gndFlag)
                    keff_coplanar_gnd2_fem[i_er, i_tmet, i_s] = keff
                    z0_coplanar_gnd2_fem[i_er, i_tmet, i_s] = z0
    data = {
        "keff_coplanar_gnd2_wc" : keff_coplanar_gnd2_wc,
        "keff_coplanar_gnd2_fem" : keff_coplanar_gnd2_fem,
        "z0_coplanar_gnd2_wc" : z0_coplanar_gnd2_wc,
        "z0_coplanar_gnd2_fem" : z0_coplanar_gnd2_fem,

        "keff_coplanar_gnd2_mswc" : keff_coplanar_gnd2_mswc,
        "keff_coplanar_gnd2_msfem" : keff_coplanar_gnd2_msfem,
        "z0_coplanar_gnd2_mswc" : z0_coplanar_gnd2_mswc,
        "z0_coplanar_gnd2_msfem" : z0_coplanar_gnd2_msfem,
    }
    print(f"Saving data to {npz}")
    np.savez_compressed(npz, **data)        
        
gndStr = "With" if gndFlag else "Without"
gndStr += " bottom GND"
for i_er in range(num_er):
    er = all_er[i_er]

    figk = plt.figure()
    ax_k = plt.subplot(111)
    ax_k.set_xscale("log")
    ax_k.grid()
    ax_k.set_title(f"Coplanar Effective Dielectric Constant\ner = {er}, W/H={W/H}, {gndStr}")
    ax_k.set_xlabel("Space/Height")
    ax_k.set_ylabel("keff")

    figz = plt.figure()
    ax_z = plt.subplot(111)
    ax_z.set_xscale("log")
    ax_z.grid()
    ax_z.set_title(f"Coplanar Characteristic Impedance\ner = {er}, W/H={W/H} {gndStr}")
    ax_z.set_xlabel("Space/Height")
    ax_z.set_ylabel("Z0 [Ohms]")
    
    for i_tmet in range(num_tmet):
        Tmet = all_tmet[i_tmet]
        sty_wc["color"] = sty_fem["color"] = colors[i_tmet % len(colors)]

        x = all_s / H
        ax_k.plot(x, keff_coplanar_gnd2_wc[i_er, i_tmet, :], label=f"WC: er={er}, Tmet={Tmet}", **sty_wc)
        ax_z.plot(x, z0_coplanar_gnd2_wc[i_er, i_tmet, :], label=f"WC: er={er}, Tmet={Tmet}", **sty_wc)
        ax_k.plot(x, keff_coplanar_gnd2_fem[i_er, i_tmet, :], label=f"FEM: er={er}, Tmet={Tmet}", **sty_fem)
        ax_z.plot(x, z0_coplanar_gnd2_fem[i_er, i_tmet, :], label=f"FEM: er={er}, Tmet={Tmet}", **sty_fem)

        if False:
            x2 = np.array([x.min(), x.max()])
            ax_k.plot(x2, [keff_coplanar_gnd2_mswc[i_er, i_tmet]]*2, **sty_wc)
            ax_z.plot(x2, [z0_coplanar_gnd2_mswc[i_er, i_tmet]]*2, **sty_wc)
            ax_k.plot(x2, [keff_coplanar_gnd2_msfem[i_er, i_tmet]]*2, **sty_fem)
            ax_z.plot(x2, [z0_coplanar_gnd2_msfem[i_er, i_tmet]]*2, **sty_fem)

    ax_k.legend()
    ax_z.legend()
    pout = save_dir / f"coplanar_{gndStr2}2_keff.png"
    figk.savefig(str(pout), dpi=dpi, bbox_inches="tight")
    pout = save_dir / f"coplanar_{gndStr2}2_z0.png"
    figz.savefig(str(pout), dpi=dpi, bbox_inches="tight")


In [ ]:
update_calc = not False
gndFlag = True
gndStr2 = "gnd" if gndFlag else "nognd"
npz = save_dir / f"coplanar_{gndStr2}3.npz"

W = 0.5*H
all_s_short = np.array([0.25*H, 0.5*H, H, 2*H])
all_tmet_swp = np.logspace(np.log10(0.001*H), np.log10(0.1*H), num_w)
num_s_short = all_s_short.shape[0]
num_tmet_swp = all_tmet_swp.shape[0]
shp3 = [num_er, num_tmet_swp, num_s_short]

if update_calc:
    keff_coplanar_gnd3_wc = np.zeros(shp3)
    z0_coplanar_gnd3_wc = np.zeros(shp3)
    keff_coplanar_gnd3_fem = np.zeros(shp3)
    z0_coplanar_gnd3_fem = np.zeros(shp3)
else:
    data = np.load(npz)
    keff_coplanar_gnd3_wc = data["keff_coplanar_gnd3_wc"]
    keff_coplanar_gnd3_fem = data["keff_coplanar_gnd3_fem"]
    z0_coplanar_gnd3_wc = data["z0_coplanar_gnd3_wc"]
    z0_coplanar_gnd3_fem = data["z0_coplanar_gnd3_fem"]

if update_calc:
    for i_er in trange(num_er, desc="Er"):
        er = all_er[i_er]    
        for i_tmet in trange(num_tmet_swp, desc="Tmet", leave=None):
            Tmet = all_tmet_swp[i_tmet]
            for i_s in trange(num_s_short, desc="S", leave=None):
                Sx = all_s_short[i_s]
                
                (keff , z0) = coplanar_calc_wcalc(W, Sx, H, Tmet, er, gndFlag)
                keff_coplanar_gnd3_wc[i_er, i_tmet, i_s] = keff
                z0_coplanar_gnd3_wc[i_er, i_tmet, i_s] = z0

                (keff , z0) = coplanar_calc_fem(W, Sx, H, Tmet, er, gndFlag)
                keff_coplanar_gnd3_fem[i_er, i_tmet, i_s] = keff
                z0_coplanar_gnd3_fem[i_er, i_tmet, i_s] = z0
    data = {
        "keff_coplanar_gnd3_wc" : keff_coplanar_gnd3_wc,
        "keff_coplanar_gnd3_fem" : keff_coplanar_gnd3_fem,
        "z0_coplanar_gnd3_wc" : z0_coplanar_gnd3_wc,
        "z0_coplanar_gnd3_fem" : z0_coplanar_gnd3_fem,
    }
    np.savez_compressed(npz, **data)        
        
gndStr = "With" if gndFlag else "Without"
gndStr += " bottom GND"
for i_er in range(num_er):
    er = all_er[i_er]

    figk = plt.figure()
    ax_k = plt.subplot(111)
    ax_k.set_xscale("log")
    ax_k.grid()
    ax_k.set_title(f"Coplanar Effective Dielectric Constant\ner = {er}, W/H={W/H}, {gndStr}")
    ax_k.set_xlabel("Tmet/Height")
    ax_k.set_ylabel("keff")

    figz = plt.figure()
    ax_z = plt.subplot(111)
    ax_z.set_xscale("log")
    ax_z.grid()
    ax_z.set_title(f"Coplanar Characteristic Impedance\ner = {er}, W/H={W/H} {gndStr}")
    ax_z.set_xlabel("Tmet/Height")
    ax_z.set_ylabel("Z0 [Ohms]")
    
    for i_s in range(num_s_short):
        Sx = all_s_short[i_s]
        sty_wc["color"] = sty_fem["color"] = colors[i_s % len(colors)]

        x = all_tmet_swp / H
        ax_k.plot(x, keff_coplanar_gnd3_wc[i_er, :, i_s], label=f"WC: er={er}, S/H={Sx/H}", **sty_wc)
        ax_z.plot(x, z0_coplanar_gnd3_wc[i_er, :, i_s], label=f"WC: er={er}, S/H={Sx/H}", **sty_wc)
        ax_k.plot(x, keff_coplanar_gnd3_fem[i_er, :, i_s], label=f"FEM: er={er}, S/H={Sx/H}", **sty_fem)
        ax_z.plot(x, z0_coplanar_gnd3_fem[i_er, :, i_s], label=f"FEM: er={er}, S/H={Sx/H}", **sty_fem)
    ax_k.legend()
    ax_z.legend()
    pout = save_dir / f"coplanar_{gndStr2}3_keff.png"
    figk.savefig(str(pout), dpi=dpi, bbox_inches="tight")
    pout = save_dir / f"coplanar_{gndStr2}3_z0.png"
    figz.savefig(str(pout), dpi=dpi, bbox_inches="tight")


In [ ]:
i_er = 2
i_tmet = 1

x = all_s / H

keff_cpw = keff_coplanar_gnd2_wc[i_er, i_tmet, :]
keff_ms = keff_coplanar_gnd2_mswc[i_er, i_tmet]
z0_cpw = z0_coplanar_gnd2_wc[i_er, i_tmet, :]
z0_ms = z0_coplanar_gnd2_mswc[i_er, i_tmet]
z0_fem = z0_coplanar_gnd2_fem[i_er, i_tmet, :]
y0_ms = 1.0 / z0_ms
y0_cpw = 1.0 / z0_cpw
m=2

d = y0_cpw/y0_ms
y = y0_ms* (1 + d**m)**(1/m)
y = (y0_ms**m + y0_cpw**m)**(1/m)
z = 1/y
zmm = np.minimum(z0_cpw, z0_ms)

k = keff_cpw
f = np.tanh(np.exp(0.5*(0.95*z0_cpw-z0_ms)))

plt.figure()
plt.plot(x,f)
plt.xscale("log")

#z = z0_ms*f + z0_cpw*(1-f)
plt.figure()
plt.plot(x,z,'r-')
plt.plot(x,z0_fem, 'b--')
plt.plot(x,z0_cpw, 'b--')
plt.plot(x,zmm, 'k-.')
plt.xscale("log")